In [1]:
import os
import sys
sys.path.append('../')
import torch
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.utils as utils
import argparse
import time
import numpy as np
import cv2
from PIL import Image, ImageOps
from photo_wct import PhotoWCT, TORCH_MODE, get_dev_vec
from photo_smooth import Propagator
from collections import namedtuple
# prepare paths for model
vgg_paths = {}
for i in range(1,6):
    vgg_paths['vgg{}'.format(i)] = '../models/vgg_normalised_conv{}_1_mask.t7'.format(i) 
    vgg_paths['decoder{}'.format(i)] = '../models/feature_invertor_conv{}_1_mask.t7'.format(i)

vgg_key_list = list(vgg_paths.keys())
vgg_arg_class = namedtuple('VggArgs', vgg_key_list)
vgg_args = vgg_arg_class(*[vgg_paths[k] for k in vgg_key_list])
# Load model
p_wct = PhotoWCT(vgg_args)
p_pro = Propagator()

def process_image(cont_img, styl_img, 
                  content_image_path, 
                  output_image_path = 'cur_image.png',
                 resize_dim = (256, 256)):
    cont_img = cont_img.resize(resize_dim, Image.BICUBIC)
    styl_img = styl_img.resize(resize_dim, Image.BICUBIC)
    cont_img = transforms.ToTensor()(cont_img).unsqueeze(0)
    styl_img = transforms.ToTensor()(styl_img).unsqueeze(0)
    cont_img = Variable(get_dev_vec(cont_img), volatile=True)
    styl_img = Variable(get_dev_vec(styl_img), volatile=True)
    
    cont_seg = []
    styl_seg = []
    cont_seg = np.asarray(cont_seg)
    styl_seg = np.asarray(styl_seg)
    
    start_style_time = time.time()
    stylized_img = p_wct.transform(cont_img, styl_img, cont_seg, styl_seg)
    end_style_time = time.time()
    print('Elapsed time in stylization: %f' % (end_style_time - start_style_time))
    new_style_image = stylized_img.data.cpu().float()
    utils.save_image(new_style_image, output_image_path, nrow=1)
    start_propagation_time = time.time()
    out_img = p_pro.process(output_image_path, content_image_path)
    end_propagation_time = time.time()
    print('Elapsed time in propagation: %f' % (end_propagation_time - start_propagation_time))
    out_img = Image.fromarray(np.array(out_img)[:,:,::-1])
    return new_style_image.numpy().squeeze().swapaxes(0,2).swapaxes(0,1), out_img

In [2]:
import ipywidgets as ipw
from glob import glob
all_images = glob(os.path.join('..', 'images', 'style', '*.png'))+glob(os.path.join('..', 'images', 'content', '*.png'))+glob(os.path.join('..', 'images', '*.png'))

In [3]:
import matplotlib.pyplot as plt
figA, (ax1, ax2) = plt.subplots(1, 2, figsize = (12, 6))
figB, (ax3, ax4) = plt.subplots(1, 2, figsize = (12, 6))
ax1.set_title('Not Loaded')
figA.savefig('junk_figure.png', dpi = 50)
def_img = open('junk_figure.png', 'rb').read()
load_image = ipw.Image(layout = ipw.Layout(width = "800px"), 
                      value = def_img)
out_image = ipw.Image(layout = ipw.Layout(width = "800px"), 
                      value = def_img)
cont_path = ipw.Dropdown(options = all_images)
style_path = ipw.Dropdown(options = all_images)
res_level = ipw.Dropdown(options = [64, 128, 256, 384], value = 64)

In [4]:
def update_image(*args):
    cont_img = Image.open(cont_path.value).convert('RGB')
    styl_img = Image.open(style_path.value).convert('RGB')
    ax1.imshow(cont_img)
    ax1.set_title('Content')
    ax2.imshow(styl_img)
    ax2.set_title('Style')
    
    figA.savefig('load_figure.png')
    load_image.value = open('load_figure.png', 'rb').read()
    out_image.value = def_img
    
    ax3.set_title('Transfered')
    resize_dim = (res_level.value, res_level.value) 
    c_img, d_img = process_image(cont_img = cont_img,
                 styl_img = styl_img,
                 content_image_path=cont_path.value, 
                                 resize_dim = resize_dim
                 )
    ax3.imshow(np.clip(c_img*255, 0, 255).astype(np.uint8))
    ax4.imshow(d_img)
    ax4.set_title('Transfered and Propogated')
    figB.savefig('t_figure.png')
    out_image.value = open('t_figure.png', 'rb').read()

cont_path.observe(update_image, names='value')
style_path.observe(update_image, names='value')
res_level.observe(update_image, names='value')

In [5]:
ipw.VBox([ipw.HBox([ipw.VBox([ipw.Label(value = 'Content File:'), cont_path]), 
                    ipw.VBox([ipw.Label(value = 'Style File:'), style_path]),
                    ipw.VBox([ipw.Label(value = 'Image Resolution:'), res_level])
                   ]), 
          ipw.VBox([load_image, out_image])])

VBox(children=(HBox(children=(VBox(children=(Label(value='Content File:'), Dropdown(options=('../images/style/in120.png', '../images/style/in90.png', '../images/style/in86.png', '../images/style/in81.png', '../images/style/in87.png', '../images/content/in120.png', '../images/content/in90.png', '../images/content/in86.png', '../images/content/in81.png', '../images/content/in87.png', '../images/content1.png', '../images/style1_small.png', '../images/style1.png', '../images/content1_small.png'), value='../images/style/in120.png'))), VBox(children=(Label(value='Style File:'), Dropdown(options=('../images/style/in120.png', '../images/style/in90.png', '../images/style/in86.png', '../images/style/in81.png', '../images/style/in87.png', '../images/content/in120.png', '../images/content/in90.png', '../images/content/in86.png', '../images/content/in81.png', '../images/content/in87.png', '../images/content1.png', '../images/style1_small.png', '../images/style1.png', '../images/content1_small.png'), value='../images/style/in120.png'))), VBox(children=(Label(value='Image Resolution:'), Dropdown(options=(64, 128, 256, 384), value=64))))), VBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x01,\x08\x06\x00\x00\x00\x1f=\xcd\x96\x00\x00\x00\x04sBIT\x08\x08\x08\x08|\x08d\x88\x00\x00\x00\tpHYs\x00\x00\x07\xb0\x00\x00\x07\xb0\x01\xd4\x82\x83b\x00\x00\x009tEXtSoftware\x00matplotlib version 2.1.1, http://matplotlib.org/\x00\xe6O\x9b\x00\x00\x12\x0cIDATx\x9c\xed\xdb\x7f\xa8\xd6w\xfd\xff\xf1\xc7\xf1lc\xa0\xceIdF\xc7q\x1aFlh,\xb6\x15ccI\xb4l\x91\xa1g\x87\xd3\xd8\xc1)\xe9Y\x1d\x99\x12\x87\x12I\x02\xa3\x86\xa6\xcbe\xfbQ\xe3\xac\xb9\xcd\x8c&x\xa2Cs^\x1b\xd5d4\x98\t\t;\x93U\x9b\x9d\xe8b+bl:%\xd3y\xae\xcf\x1f}v\xbe\x1c\xdc\xe9\xb3\xaf{\x9d\xf3\xf6\xec\xdcn\xff\x9d\xf7\xf1:>\x10\x9ep\xe7\xf2:M\x8dF\xa3\x11\x00\x00\x8a\x99R\xf5\x00\x00\x80\xf7\x1a\x81\x05\x00P\x98\xc0\x02\x00(L`\x01\x00\x14&\xb0\x00\x00\n\x13X\x00\x00\x85\t,\x00\x80\xc2\x04\x16\x00@a\x02\x0b\x00\xa00\x81\x05\x00P\x98\xc0\x02\x00(L`\x01\x00\x14&\xb0\x00\x00\n\x13X\x00\x00\x85\t,\x00\x80\xc2\x04\x16\x00@a\x02\x0b\x00\xa00\x81\x05\x00P\x98\xc0\x02\x00(L`\x01\x00\x14&\xb0\x00\x00\n\x13X\x00\x00\x85\t,\x00\x80\xc2\x04\x16\x00@a\x02\x8bq788\x98\xa6\xa6\xa6\xec\xdf\xbf?I\xf2\xab_\xfd*\x1b6l\x18\xf5\xcf?\xf4\xd0C9y\xf2\xe4\x19\xcf\x17,X\x90c\xc7\x8e\xbd\xeb=W]u\xd5;\xfas\xff\xd7N\x00x\x8b\xc0\xa2\x12\x97_~y6o\xde\xfc\x8e\xfe\xech\x81\x05\x00\xe7\xaa\xf3\xaa\x1e\xc0\xe4t\xd9e\x97\xe5\xcd7\xdf\xcc\x9f\xfe\xf4\xa7\x11\xcf\x7f\xfe\xf3\x9f\xe7\xae\xbb\xeeJSSS\xbe\xfd\xedo\xe7\xa2\x8b.\xca\xc1\x83\x07s\xe3\x8d7f\xc9\x92%\xe9\xe9\xe9\x19\xf5g\x9e>}:\xcb\x96-\xcb\xdf\xfe\xf6\xb7L\x9b6-?\xfd\xe9O3c\xc6\x8c|\xf6\xb3\x9f\xcd\xa9S\xa7r\xc1\x05\x17d\xf7\xee\xdd\xb9\xe8\xa2\x8b\xb2i\xd3\xa6\xfc\xf2\x97\xbf\xccUW]\x95\xa1\xa1\xa1$\xc9\xe1\xc3\x87\xd3\xdd\xdd\x9d\x7f\xff\xfb\xdf\xf9\xf8\xc7?\x9e\xbb\xee\xba+G\x8e\x1c\xc9\x97\xbe\xf4\xa5$\xc9\xec\xd9\xb3\xd3\xda\xda:f\xff&\x00\xbcwx\x07\x8b\xca|\xfd\xeb_\xcf\x96-[\x86\xbf>}\xfat6n\xdc\x98}\xfb\xf6\xe5\x89\'\x9e\xc8\xfa\xf5\xebs\xcd5\xd7\xe4\x8a+\xae\xc8\xe3\x8f?\xfe_\xe3*I~\xf1\x8b_\xa4\xa5\xa5%\xfb\xf6\xed\xcb\xcd7\xdf\x9c\xbb\xef\xbe;S\xa6LI\x7f\x7f\x7f\xf6\xed\xdb\x97\xcf\x7f\xfe\xf3y\xf4\xd1G\xf3\x8f\x7f\xfc#\x8f?\xfex\x9ey\xe6\x99\xacZ\xb5*\xaf\xbd\xf6Z\x92d\xdd\xbau\xb9\xef\xbe\xfb\xf2\xd4SO\xe5\xc4\x89\x139p\xe0@z{{\xd3\xd6\xd6\x96\xbd{\xf7f\xf6\xec\xd9c\xfa\xef\x01\xc0{\x87\xc0\xa22\xd7]w]^z\xe9\xa5\xbc\xf2\xca+I\x92\x7f\xfe\xf3\x9f\xb9\xe4\x92Kr\xe1\x85\x17\xe6\xa2\x8b.\xca\xf9\xe7\x9f\x9f7\xdf|\xf3\x1d\xff\xbc\x17_|1W_}u\x92\xe4\xea\xab\xaf\xce\x9f\xff\xfc\xe7\x1c;v,]]]\xf9\xd4\xa7>\x95\x07\x1f|0/\xbf\xfcr\xfe\xf2\x97\xbf\xe4c\x1f\xfbX\x9a\x9a\x9ar\xd9e\x97e\xda\xb4iI\x92\x17^x!+V\xac\xc8\x82\x05\x0b\xb2\x7f\xff\xfe\xd4\xeb\xf5\xbc\xf8\xe2\x8b\xb9\xf2\xca+\x87\x7f&\x00\xbc\x13\x02\x8bJ}\xedk_\xcb\x0f\x7f\xf8\xc3$\xc9\xfb\xdf\xff\xfe\xfc\xf5\xaf\x7f\xcd\x89\x13\'r\xf4\xe8\xd1\x9c<y2\xe7\x9dw^\xce?\xff\xfc\x9c>}\xfa\xff\xfcY

Elapsed time in stylization: 0.754482
Elapsed time in propagation: 0.148189
Elapsed time in stylization: 2.280497
Elapsed time in propagation: 0.762288
Elapsed time in stylization: 2.165730
Elapsed time in propagation: 0.710015
